# Step 1 to create GOES data timeseries

- Creates files with lists of all GOES files in 1 day

- Run this before creating GOES timeseries data

- Set start date

- Set number of days to increment through (does 1 day at a time)

This primarily supports creating the historical GOES data. This has to access 288 files per day, and with all those requests, there is risk of failure. Having a static list of filenames to go through was a quick solution to be able to make sure we started again at the same place we ended when the program died.


In [1]:
#create goes data
from goes2go import GOES
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#################  SET THESE VALUES ######################

start_year, start_month, start_day = 2023,12,6
number_days_to_process = 9

##########################################################


def get_start_end_time(fname):
    #goes filenames structure https://geonetcast.wordpress.com/2017/04/27/goes-16-file-naming-convention/
    #use filename to find start/end times for data
    tem = str(fname).split('/')
    tem2,i = tem[5],25
    dt_start = datetime.strptime(tem2[i:i+13], '%Y%j%H%M%S')
    tem2,i = tem[5],41
    dt_end = datetime.strptime(tem2[i:i+13], '%Y%j%H%M%S')
    return dt_start,dt_end


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/goes2go/data.py:665: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/goes2go/NEW.py:188: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),


In [3]:
now = datetime.now()
end_time = now.strftime("%Y-%m-%d %H:%M")
yesterday = now - timedelta(hours=24)
start_time = yesterday.strftime("%Y-%m-%d %H:%M")

for incr in range(0,number_days_to_process):

    date_start = datetime(start_year, start_month, start_day) + timedelta(days=incr)
    date_end = date_start + timedelta(days=1)  
    start_time = date_start.strftime("%Y%m%d")
    end_time = date_end.strftime("%Y%m%d")
    
    print(start_time,end_time)
    
    # Open the GOES-R image
    G = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')  #ABI-L2-DMWVC ABI-L2-DMWC not at all points #
    # Produce a pandas DataFrame of the available files in a time range
    df = G.df(start=start_time, end=end_time)
    print(df.file[0])
    # Save DataFrame to a CSV file
    files_fname = './data/goes_filenames_test_'+start_time+'-'+end_time+'.csv'
    df.to_csv(files_fname)


20231206 20231207
noaa-goes16/ABI-L2-MCMIPC/2023/340/00/OR_ABI-L2-MCMIPC-M6_G16_s20233400001173_e20233400003546_c20233400004056.nc
20231207 20231208
noaa-goes16/ABI-L2-MCMIPC/2023/341/00/OR_ABI-L2-MCMIPC-M6_G16_s20233410001174_e20233410003558_c20233410004065.nc
20231208 20231209
noaa-goes16/ABI-L2-MCMIPC/2023/342/00/OR_ABI-L2-MCMIPC-M6_G16_s20233420001174_e20233420003547_c20233420004068.nc
20231209 20231210
noaa-goes16/ABI-L2-MCMIPC/2023/343/00/OR_ABI-L2-MCMIPC-M6_G16_s20233430001175_e20233430003547_c20233430004064.nc
20231210 20231211
noaa-goes16/ABI-L2-MCMIPC/2023/344/00/OR_ABI-L2-MCMIPC-M6_G16_s20233440001175_e20233440003560_c20233440004066.nc
20231211 20231212
noaa-goes16/ABI-L2-MCMIPC/2023/345/00/OR_ABI-L2-MCMIPC-M6_G16_s20233450001176_e20233450003560_c20233450004075.nc
20231212 20231213
noaa-goes16/ABI-L2-MCMIPC/2023/346/00/OR_ABI-L2-MCMIPC-M6_G16_s20233460001170_e20233460003549_c20233460004061.nc
20231213 20231214
noaa-goes16/ABI-L2-MCMIPC/2023/347/00/OR_ABI-L2-MCMIPC-M6_G16_s20